In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

In [2]:
# Download training data from open datasets.
# Guassian blur - how to do: RESEARCH
training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(), transforms.RandomHorizontalFlip(),
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                    transforms.RandomVerticalFlip(), transforms.RandomRotation(45)]),
)

# Download test data from open datasets.
test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]),
)

# Download test data from open datasets.
val_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]),
)

100%|██████████| 344862509/344862509 [00:17<00:00, 20169995.11it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 360291.00it/s]


100%|██████████| 14989/14989 [00:00<00:00, 15677910.89it/s]


In [3]:
batch_size = 32
num_classes = 102
learning_rate = 0.001
num_epochs = 90

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
val_dataloader = DataLoader(val_data, batch_size=batch_size)

In [4]:
import torch.nn.functional as F
import os

class Net(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.batch_layer1 = nn.BatchNorm2d(num_features=64) 
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.batch_layer2 = nn.BatchNorm2d(num_features=128)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        self.batch_layer3 = nn.BatchNorm2d(num_features=256)
        self.max_pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3)
        self.batch_layer4 = nn.BatchNorm2d(num_features=512)
        self.max_pool4 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(2048 , 128)
        self.relu1 = nn.PReLU()
        self.fc2 = nn.Linear(128, num_classes)
    

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.batch_layer1(out)
        out = self.relu1(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer2(out)
        out = self.batch_layer2(out)
        out = self.relu1(out)
        out = self.max_pool2(out)

        out = self.conv_layer3(out)
        out = self.batch_layer3(out)
        out = self.relu1(out)
        out = self.max_pool3(out)

        out = self.conv_layer4(out)
        out = self.batch_layer4(out)
        out = self.relu1(out)
        out = self.max_pool4(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

net = Net(num_classes)

def save_network(network, epoch_label):
    save_filename = 'net_%s.pth' % epoch_label
    save_path = os.path.join(save_filename)
    torch.save(network.state_dict(), save_path)

def testValidationData(network):
    net.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        return (100 * correct / total)

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
saveCounter = 0
minLoss = 10000
valAccurary = 0


# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_dataloader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
      
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if minLoss > loss and epoch % 5 == 0:
        tempAcc = testValidationData(net)
        if tempAcc > valAccurary:
            valAccurary = tempAcc
            saveCounter += 1
            save_network(net, saveCounter)
            print("SAVE COMPLETE NEW ACC:", valAccurary)


    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

SAVE COMPLETE NEW ACC: 6.2745098039215685
Epoch [1/90], Loss: 4.0828
Epoch [2/90], Loss: 3.7023
Epoch [3/90], Loss: 3.4363
Epoch [4/90], Loss: 3.2839
Epoch [5/90], Loss: 3.0160
SAVE COMPLETE NEW ACC: 22.647058823529413
Epoch [6/90], Loss: 2.9840
Epoch [7/90], Loss: 2.8204
Epoch [8/90], Loss: 2.4316
Epoch [9/90], Loss: 2.1162
Epoch [10/90], Loss: 2.1373
SAVE COMPLETE NEW ACC: 28.823529411764707
Epoch [11/90], Loss: 2.2772
Epoch [12/90], Loss: 2.0844
Epoch [13/90], Loss: 1.6832
Epoch [14/90], Loss: 1.4863
Epoch [15/90], Loss: 1.7799
SAVE COMPLETE NEW ACC: 33.627450980392155
Epoch [16/90], Loss: 1.2369
Epoch [17/90], Loss: 1.4267
Epoch [18/90], Loss: 0.6621
Epoch [19/90], Loss: 1.2065
Epoch [20/90], Loss: 0.8023
SAVE COMPLETE NEW ACC: 34.31372549019608
Epoch [21/90], Loss: 1.0791
Epoch [22/90], Loss: 0.6508
Epoch [23/90], Loss: 0.7988
Epoch [24/90], Loss: 1.1802
Epoch [25/90], Loss: 1.3258
SAVE COMPLETE NEW ACC: 37.15686274509804
Epoch [26/90], Loss: 0.7332
Epoch [27/90], Loss: 0.7845
Epo

In [7]:
net.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))

Accuracy of the network on the 6149 train images: 36.29858513579444 %
